# Phase 1: Residential Building Consent Dataset

## INPUTS
1.	Geocoded individual consent data  
Various information (specified below under OUTPUTS) need to be added to each observation in these two datasets 
-	csv file containing building consents for 2000 to 2020 [filename: BCs_issued_by_AUP_TLADCs_2021FEB.csv]
2.	LINZ spine from phase 0

## OUTPUTS
csv file of building consents with the data fields 1-9 above. The matching process for assigning each consent to a LINZ parcel is described below. 
The following additional data fields are added. These are flags to designate the match to the LINZ dataset.

10.	Ranged Address Indicator  
a.	Indicator (1 or 0) for LINZ parcel that is part of a ranged address consent Ranged_Address_indicator

In [1]:
#pragma nodebook off
#Use nodebook for better reproducibility https://github.com/uoa-eResearch/nodebook
%reload_ext nodebook.ipython
%nodebook disk phase1

<IPython.core.display.Javascript object>

In [ ]:
# load libraries
import geopandas as gpd # vector data
import pandas as pd # tabular data, loading CSVs
import numpy as np # numeric data
from util import *
import matplotlib # plotting
import contextily as ctx # Used for contextual basemaps
from matplotlib_scalebar.scalebar import ScaleBar # scalebar for plot
import matplotlib.pyplot as plt # plotting
from tqdm.auto import tqdm # progress bars
tqdm.pandas()
import json
from shapely.geometry import Point, shape, LineString, MultiLineString, GeometryCollection, MultiPoint, Polygon  # creating points
plt.rcParams['figure.figsize'] = (20, 20)
pd.set_option('max_columns', None)
pd.set_option('max_rows', 300)

/usr/local/lib/python3.8/dist-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
%%time
df = pd.read_csv("restricted/BCs_issued_by_AUP_TLADCs_2021FEB.csv", encoding='cp1252')
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.X_Coordinate, df.Y_Coordinate), crs=4326)

CPU times: user 1.34 s, sys: 82.3 ms, total: 1.43 s
Wall time: 1.44 s


In [ ]:
df

,OBS,CONSENT_DATE,MARCH_YEAR,ADDRESS_1,ADDRESS_2,ADDRESS_3,LB_Name,BUILDING_TYPE_NAME,BUILDING_TYPE_CODE,FLOOR_AREA,VALUE,BUILDINGS,Building_Type_Group,Residential_Type,Business_Group,Business_Category,Type_Class_Year,MBCODE,MBYEAR,AUP_BaseZone,AUP_BaseZone_Group,ZONE,ZONE_ID,ZONE_NAME,ZONE_TYPE,BUSINESS_TYPE,PlanAreaName,SHA_Name_154,X_Coordinate,Y_Coordinate,geometry
0,227614,"Feb 01, 2021 12:00:00 AM","Mar 01, 2020 12:00:00 AM",398 PAKIRI BLOCK ROAD,Tomarata,Auckland,Rodney,Houses,1111,231.0,NaN,1,Residential,Detached,NaN,NaN,2014,135900.0,2013.0,Road,General,27.0,NaN,NaN,NaN,NaN,NaN,NaN,174.634550,-36.224858,POINT (174.63455 -36.22486)
1,227656,"Feb 01, 2021 12:00:00 AM","Mar 01, 2020 12:00:00 AM",Ahuroa Road,Puhoi,Auckland,Rodney,Non-building construction,3111,0.0,NaN,1,Other,NaN,NaN,NaN,2014,146701.0,2013.0,Road,General,27.0,NaN,NaN,NaN,NaN,NaN,NaN,174.568721,-36.475863,POINT (174.56872 -36.47586)
2,227796,"Feb 01, 2021 12:00:00 AM","Mar 01, 2020 12:00:00 AM",Upper Orewa Road,Upper Orewa,Auckland,Rodney,"Townhouses, flats, units, and other dwellings",1129,260.0,NaN,2,Residential,Attached,NaN,NaN,2014,177701.0,2013.0,Road,General,27.0,NaN,NaN,NaN,NaN,NaN,NaN,174.641586,-36.590190,POINT (174.64159 -36.59019)
3,227896,"Feb 01, 2021 12:00:00 AM","Mar 01, 2020 12:00:00 AM",56 VINISTRA ROAD,Huapai,Auckland,Rodney,Houses,1111,179.0,NaN,1,Residential,Detached,NaN,NaN,2014,214900.0,2013.0,Road,General,27.0,RDC_7,Future Urban,Rural,NaN,Huapai Triangle,Huapai Triangle,174.541738,-36.778192,POINT (174.54174 -36.77819)
4,227902,"Feb 01, 2021 12:00:00 AM","Mar 01, 2020 12:00:00 AM",77 VINISTRA ROAD,Huapai,Auckland,Rodney,Houses,1111,180.0,NaN,1,Residential,Detached,NaN,NaN,2014,214900.0,2013.0,Road,General,27.0,RDC_7,Future Urban,Rural,NaN,Huapai Triangle,Huapai Triangle,174.541738,-36.778192,POINT (174.54174 -36.77819)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207221,151352,"Jul 01, 2014 12:00:00 AM","Mar 01, 2015 12:00:00 AM",164-168,HOBSON ST,AUCKLAND,Waitemata,New Flats,1200,0.0,3650000.0,0,Residential,Attached,NaN,NaN,1998,434200.0,2013.0,Business - City Centre Zone,Business,35.0,CBD_1,Central Area A,Business,Commercial,NaN,NaN,174.760314,-36.851526,POINT (174.76031 -36.85153)
207222,151351,"Jul 01, 2014 12:00:00 AM","Mar 01, 2015 12:00:00 AM",464,REMUERA RD,REMUERA,Orakei,New Flats,1200,0.0,120000.0,0,Residential,Attached,NaN,NaN,1998,490600.0,2013.0,Residential - Terrace Housing and Apartment Bu...,Residential,8.0,ACC_28,Residential 7c,Residential,NaN,NaN,NaN,174.801622,-36.880545,POINT (174.80162 -36.88055)
207223,151350,"Jul 01, 2014 12:00:00 AM","Mar 01, 2015 12:00:00 AM",55B,PANORAMA RD,MOUNT WELLINGTON,Maungakiekie - Tamaki,"New (and pre-built) house, unit, bach, crib, b...",1100,140.0,200000.0,1,Residential,Detached,NaN,NaN,1998,643401.0,2013.0,Residential - Mixed Housing Suburban Zone,Residential,18.0,ACC_24,Residential 6a,Residential,NaN,NaN,NaN,174.830461,-36.904046,POINT (174.83046 -36.90405)
207224,151349,"Jul 01, 2014 12:00:00 AM","Mar 01, 2015 12:00:00 AM",67,CUSTOMS ST EAST,AUCKLAND CENTRAL,Waitemata,New Offices,1461,0.0,0.0,1,Business,NaN,Group 2,Offices,1998,438104.0,2013.0,Business - City Centre Zone,Business,35.0,CBD_2,Central Area B,Business,Commercial,NaN,NaN,174.768483,-36.845564,POINT (174.76848 -36.84556)


In [ ]:
df["Ranged_Address_indicator"] = df.ADDRESS_1.str.contains('\d\w?\s?-', regex=True, na=False)
df["Ranged_Address_indicator"].value_counts()

False    200848
True       6378
Name: Ranged_Address_indicator, dtype: int64

11.	Matching Type Indicators:  
a.	LINZ_MATCH_CODE  
b.	LINZ_2ND_MATCH_CODE  
PROCEDURE FOR MATCHING CONSENTS TO LINZ PARCELS  
MATCHING FOR NON-RANGED ADDRESSES:  
   Find the LINZ parcel of the geo-coordinate of the consent.  
a. Check whether the address of the consent and the address of the LINZ parcel match, by matching on number and first word. If so, stop and set LINZ_MATCH_CODE = 1. If not, go on to 2:

In [ ]:
%%time
# get number and name of street (but not 'road', 'street', 'place' etc)
# this can be used to match addresses with building consents

def number_name_bc(x):
    """extract street number and first complete word of the street name from building consents"""
    if x.ADDRESS_1 is None:
        pass
    else:
        # get number and first word of address
        joined_address = ' '.join([str(x[f'ADDRESS_{i}']) for i in [1,2, 3]]).lower()
        return ' '.join(joined_address.split(' ')[:2])

def full_address_bc(x):
    """extract full address from building consents"""
    if x.ADDRESS_1 is None:
        pass
    else:
        # get number and first word of address
        joined_address = ' '.join([str(x[f'ADDRESS_{i}']) for i in [1,2, 3] if not str(x[f'ADDRESS_{i}']) == 'nan']).lower()
        return joined_address

df['number_name'] = df.apply(number_name_bc, axis=1)
df['full_address'] = df.apply(full_address_bc, axis=1)

CPU times: user 22.4 s, sys: 39.9 ms, total: 22.4 s
Wall time: 22.4 s


In [ ]:
parcels = gpd.read_file('input/lds-nz-primary-parcels-FGDB.zip!nz-primary-parcels.gdb').to_crs(2193)
parcels = parcels.set_index("id")
parcels

,appellation,affected_surveys,parcel_intent,topology_type,statutory_actions,land_district,titles,survey_area,calc_area,geometry
id,,,,,,,,,,
5266447,None,None,Hydro,Primary,None,North Auckland,None,NaN,1289690.0,"MULTIPOLYGON (((1731236.079 5985947.435, 17312..."
4789727,Part Lot 3 Allot 64 Section 1 SBRS OF Auckland,SO 663,DCDB,Primary,None,North Auckland,None,NaN,1.0,"MULTIPOLYGON (((1758545.404 5920084.329, 17585..."
4810316,Part Tidal Lands of Manukau Harbour Survey Off...,SO 67474,DCDB,Primary,[Create] Revested in the Crown Sec 5 Foreshore...,North Auckland,None,31600000.0,31342451.0,"MULTIPOLYGON (((1750198.000 5901985.621, 17502..."
4817943,Crown Land Survey Office Plan 58175,SO 58175,DCDB,Primary,[Create] Crown Land Reserved from Sale Sec 58 ...,North Auckland,None,NaN,467490.0,"MULTIPOLYGON (((1719456.228 5978851.350, 17194..."
4827816,Lot 2 DP 165098,DP 165098,DCDB,Primary,None,North Auckland,NA99B/977,22979.0,22972.0,"MULTIPOLYGON (((1754915.994 5937843.620, 17549..."
...,...,...,...,...,...,...,...,...,...,...
7291940,Lot 745 DP 433546,DP 433546,Fee Simple Title,Primary,None,North Auckland,528968,247.0,247.0,"MULTIPOLYGON (((1772345.368 5899041.420, 17723..."
7266269,Lot 533 DP 427884,"DP 427884, LT 454543",Fee Simple Title,Primary,None,North Auckland,520875,336.0,336.0,"MULTIPOLYGON (((1772527.587 5899136.912, 17725..."
8051540,Lot 8 DP 533517,DP 533517,Fee Simple Title,Primary,None,North Auckland,876934,185.0,185.0,"MULTIPOLYGON (((1749626.295 5949044.689, 17496..."


In [ ]:
addresses = gpd.read_file('input/lds-nz-street-address-GPKG-CLIPPED.gpkg').to_crs(2193)
addresses

,address_id,change_id,address_type,unit_value,address_number,address_number_suffix,address_number_high,water_route_name,water_name,suburb_locality,town_city,full_address_number,full_road_name,full_address,road_section_id,gd2000_xcoord,gd2000_ycoord,water_route_name_ascii,water_name_ascii,suburb_locality_ascii,town_city_ascii,full_road_name_ascii,full_address_ascii,geometry
0,2045796,2863655,Road,None,26,A,NaN,None,None,Kaukapakapa,None,26A,Henley Road,"26A Henley Road, Kaukapakapa",379281,174.497848,-36.630429,None,None,Kaukapakapa,None,Henley Road,"26A Henley Road, Kaukapakapa",POINT (1733919.256 5945079.452)
1,2045797,2863656,Road,None,26,B,NaN,None,None,Kaukapakapa,None,26B,Henley Road,"26B Henley Road, Kaukapakapa",379281,174.497236,-36.630864,None,None,Kaukapakapa,None,Henley Road,"26B Henley Road, Kaukapakapa",POINT (1733863.768 5945032.094)
2,2046395,2863772,Road,None,34,B,NaN,None,None,Manly,Whangaparaoa,34B,Rawhiti Road,"34B Rawhiti Road, Manly, Whangaparaoa",346018,174.760415,-36.632911,None,None,Manly,Whangaparaoa,Rawhiti Road,"34B Rawhiti Road, Manly, Whangaparaoa",POINT (1757391.717 5944405.691)
3,2010077,1897733,Road,1,31,None,NaN,None,None,Birkenhead,Auckland,1/31,Rawene Road,"1/31 Rawene Road, Birkenhead, Auckland",175717,174.725070,-36.813514,None,None,Birkenhead,Auckland,Rawene Road,"1/31 Rawene Road, Birkenhead, Auckland",POINT (1753870.590 5924426.522)
4,2094408,2871149,Road,None,126,B,NaN,None,None,Shelly Park,Auckland,126B,Sandspit Road,"126B Sandspit Road, Shelly Park, Auckland",304746,174.944939,-36.909427,None,None,Shelly Park,Auckland,Sandspit Road,"126B Sandspit Road, Shelly Park, Auckland",POINT (1773267.418 5913408.547)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608055,2239121,5409212,Road,None,8,A,NaN,None,None,Glendowie,Auckland,8A,Chelmsford Avenue,"8A Chelmsford Avenue, Glendowie, Auckland",163266,174.880244,-36.859001,None,None,Glendowie,Auckland,Chelmsford Avenue,"8A Chelmsford Avenue, Glendowie, Auckland",POINT (1767613.600 5919118.800)
608056,2240679,5407677,Road,None,33,A,NaN,None,None,Papatoetoe,Auckland,33A,Grange Road,"33A Grange Road, Papatoetoe, Auckland",163263,174.850682,-36.961717,None,None,Papatoetoe,Auckland,Grange Road,"33A Grange Road, Papatoetoe, Auckland",POINT (1764756.900 5907774.254)
608057,2242016,5408396,Road,None,40,A,NaN,None,None,Glen Eden,Auckland,40A,Leo Street,"40A Leo Street, Glen Eden, Auckland",375960,174.638270,-36.919333,None,None,Glen Eden,Auckland,Leo Street,"40A Leo Street, Glen Eden, Auckland",POINT (1745926.238 5912822.878)
608058,2242017,5408403,Road,None,40,B,NaN,None,None,Glen Eden,Auckland,40B,Leo Street,"40B Leo Street, Glen Eden, Auckland",375960,174.638435,-36.919365,None,None,Glen Eden,Auckland,Leo Street,"40B Leo Street, Glen Eden, Auckland",POINT (1745940.912 5912818.987)


In [ ]:
def number_name_addresses(x):
    return ' '.join(x.split(' ')[:2]).lower()
addresses['address_number_name'] = addresses.full_address.apply(number_name_addresses)
addresses

,address_id,change_id,address_type,unit_value,address_number,address_number_suffix,address_number_high,water_route_name,water_name,suburb_locality,town_city,full_address_number,full_road_name,full_address,road_section_id,gd2000_xcoord,gd2000_ycoord,water_route_name_ascii,water_name_ascii,suburb_locality_ascii,town_city_ascii,full_road_name_ascii,full_address_ascii,geometry,address_number_name
0,2045796,2863655,Road,None,26,A,NaN,None,None,Kaukapakapa,None,26A,Henley Road,"26A Henley Road, Kaukapakapa",379281,174.497848,-36.630429,None,None,Kaukapakapa,None,Henley Road,"26A Henley Road, Kaukapakapa",POINT (1733919.256 5945079.452),26a henley
1,2045797,2863656,Road,None,26,B,NaN,None,None,Kaukapakapa,None,26B,Henley Road,"26B Henley Road, Kaukapakapa",379281,174.497236,-36.630864,None,None,Kaukapakapa,None,Henley Road,"26B Henley Road, Kaukapakapa",POINT (1733863.768 5945032.094),26b henley
2,2046395,2863772,Road,None,34,B,NaN,None,None,Manly,Whangaparaoa,34B,Rawhiti Road,"34B Rawhiti Road, Manly, Whangaparaoa",346018,174.760415,-36.632911,None,None,Manly,Whangaparaoa,Rawhiti Road,"34B Rawhiti Road, Manly, Whangaparaoa",POINT (1757391.717 5944405.691),34b rawhiti
3,2010077,1897733,Road,1,31,None,NaN,None,None,Birkenhead,Auckland,1/31,Rawene Road,"1/31 Rawene Road, Birkenhead, Auckland",175717,174.725070,-36.813514,None,None,Birkenhead,Auckland,Rawene Road,"1/31 Rawene Road, Birkenhead, Auckland",POINT (1753870.590 5924426.522),1/31 rawene
4,2094408,2871149,Road,None,126,B,NaN,None,None,Shelly Park,Auckland,126B,Sandspit Road,"126B Sandspit Road, Shelly Park, Auckland",304746,174.944939,-36.909427,None,None,Shelly Park,Auckland,Sandspit Road,"126B Sandspit Road, Shelly Park, Auckland",POINT (1773267.418 5913408.547),126b sandspit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608055,2239121,5409212,Road,None,8,A,NaN,None,None,Glendowie,Auckland,8A,Chelmsford Avenue,"8A Chelmsford Avenue, Glendowie, Auckland",163266,174.880244,-36.859001,None,None,Glendowie,Auckland,Chelmsford Avenue,"8A Chelmsford Avenue, Glendowie, Auckland",POINT (1767613.600 5919118.800),8a chelmsford
608056,2240679,5407677,Road,None,33,A,NaN,None,None,Papatoetoe,Auckland,33A,Grange Road,"33A Grange Road, Papatoetoe, Auckland",163263,174.850682,-36.961717,None,None,Papatoetoe,Auckland,Grange Road,"33A Grange Road, Papatoetoe, Auckland",POINT (1764756.900 5907774.254),33a grange
608057,2242016,5408396,Road,None,40,A,NaN,None,None,Glen Eden,Auckland,40A,Leo Street,"40A Leo Street, Glen Eden, Auckland",375960,174.638270,-36.919333,None,None,Glen Eden,Auckland,Leo Street,"40A Leo Street, Glen Eden, Auckland",POINT (1745926.238 5912822.878),40a leo
608058,2242017,5408403,Road,None,40,B,NaN,None,None,Glen Eden,Auckland,40B,Leo Street,"40B Leo Street, Glen Eden, Auckland",375960,174.638435,-36.919365,None,None,Glen Eden,Auckland,Leo Street,"40B Leo Street, Glen Eden, Auckland",POINT (1745940.912 5912818.987),40b leo


In [ ]:
%%time
parcels_addressed = gpd.sjoin(parcels, addresses)

CPU times: user 46.9 s, sys: 2.59 s, total: 49.5 s
Wall time: 49.9 s


In [ ]:
%%time
parcel_address_number_name = parcels_addressed.groupby('id')['address_number_name'].apply(list)
parcel_address_number_name

/home/nyou045/git/nodebook/nodebook/pickledict.py:38: FutureWarning: to_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  return msgpack.ExtType(PANDAS_CODE, obj.to_msgpack())


CPU times: user 35.3 s, sys: 1.22 s, total: 36.6 s
Wall time: 36.7 s


id
575533          [8 aorere]
576109           [230 old]
579434        [68 opaheke]
579435        [2 boundary]
579774         [8 donovan]
                ...       
8109029         [13b ross]
8109030         [13a ross]
8109059          [1 keiha]
8109060    [17 mcwhirters]
8110496         [88 boord]
Name: address_number_name, Length: 432544, dtype: object

In [ ]:
parcels["addresses"] = parcel_address_number_name

/usr/local/lib/python3.8/dist-packages/msgpack/__init__.py:58: FutureWarning: The read_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  return unpackb(data, **kwargs)


In [ ]:
%%time
df = gpd.sjoin(df.to_crs(2193), parcels[["addresses","geometry"]], how="left")

CPU times: user 29.4 s, sys: 1.22 s, total: 30.6 s
Wall time: 30.8 s


In [ ]:
df

,OBS,CONSENT_DATE,MARCH_YEAR,ADDRESS_1,ADDRESS_2,ADDRESS_3,LB_Name,BUILDING_TYPE_NAME,BUILDING_TYPE_CODE,FLOOR_AREA,VALUE,BUILDINGS,Building_Type_Group,Residential_Type,Business_Group,Business_Category,Type_Class_Year,MBCODE,MBYEAR,AUP_BaseZone,AUP_BaseZone_Group,ZONE,ZONE_ID,ZONE_NAME,ZONE_TYPE,BUSINESS_TYPE,PlanAreaName,SHA_Name_154,X_Coordinate,Y_Coordinate,geometry,Ranged_Address_indicator,number_name,full_address,index_right,addresses
0,227614,"Feb 01, 2021 12:00:00 AM","Mar 01, 2020 12:00:00 AM",398 PAKIRI BLOCK ROAD,Tomarata,Auckland,Rodney,Houses,1111,231.0,NaN,1,Residential,Detached,NaN,NaN,2014,135900.0,2013.0,Road,General,27.0,NaN,NaN,NaN,NaN,NaN,NaN,174.634550,-36.224858,POINT (1746904.853 5989872.553),False,398 pakiri,398 pakiri block road tomarata auckland,5209817.0,NaN
1,227656,"Feb 01, 2021 12:00:00 AM","Mar 01, 2020 12:00:00 AM",Ahuroa Road,Puhoi,Auckland,Rodney,Non-building construction,3111,0.0,NaN,1,Other,NaN,NaN,NaN,2014,146701.0,2013.0,Road,General,27.0,NaN,NaN,NaN,NaN,NaN,NaN,174.568721,-36.475863,POINT (1740536.028 5962125.407),False,ahuroa road,ahuroa road puhoi auckland,5227114.0,NaN
2,227796,"Feb 01, 2021 12:00:00 AM","Mar 01, 2020 12:00:00 AM",Upper Orewa Road,Upper Orewa,Auckland,Rodney,"Townhouses, flats, units, and other dwellings",1129,260.0,NaN,2,Residential,Attached,NaN,NaN,2014,177701.0,2013.0,Road,General,27.0,NaN,NaN,NaN,NaN,NaN,NaN,174.641586,-36.590190,POINT (1746847.868 5949333.496),False,upper orewa,upper orewa road upper orewa auckland,6605265.0,NaN
3,227896,"Feb 01, 2021 12:00:00 AM","Mar 01, 2020 12:00:00 AM",56 VINISTRA ROAD,Huapai,Auckland,Rodney,Houses,1111,179.0,NaN,1,Residential,Detached,NaN,NaN,2014,214900.0,2013.0,Road,General,27.0,RDC_7,Future Urban,Rural,NaN,Huapai Triangle,Huapai Triangle,174.541738,-36.778192,POINT (1737579.970 5928624.665),False,56 vinistra,56 vinistra road huapai auckland,7859219.0,NaN
4,227902,"Feb 01, 2021 12:00:00 AM","Mar 01, 2020 12:00:00 AM",77 VINISTRA ROAD,Huapai,Auckland,Rodney,Houses,1111,180.0,NaN,1,Residential,Detached,NaN,NaN,2014,214900.0,2013.0,Road,General,27.0,RDC_7,Future Urban,Rural,NaN,Huapai Triangle,Huapai Triangle,174.541738,-36.778192,POINT (1737579.970 5928624.665),False,77 vinistra,77 vinistra road huapai auckland,7859219.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207221,151352,"Jul 01, 2014 12:00:00 AM","Mar 01, 2015 12:00:00 AM",164-168,HOBSON ST,AUCKLAND,Waitemata,New Flats,1200,0.0,3650000.0,0,Residential,Attached,NaN,NaN,1998,434200.0,2013.0,Business - City Centre Zone,Business,35.0,CBD_1,Central Area A,Business,Commercial,NaN,NaN,174.760314,-36.851526,POINT (1756936.812 5920152.000),True,164-168 hobson,164-168 hobson st auckland,6752833.0,"(37 cook, 1305/168 hobson, 1304/168 hobson, 12..."
207222,151351,"Jul 01, 2014 12:00:00 AM","Mar 01, 2015 12:00:00 AM",464,REMUERA RD,REMUERA,Orakei,New Flats,1200,0.0,120000.0,0,Residential,Attached,NaN,NaN,1998,490600.0,2013.0,Residential - Terrace Housing and Apartment Bu...,Residential,8.0,ACC_28,Residential 7c,Residential,NaN,NaN,NaN,174.801622,-36.880545,POINT (1760559.126 5916863.750),False,464 remuera,464 remuera rd remuera,4928237.0,"(1/464 remuera, 3b/464 remuera, 3a/464 remuera..."
207223,151350,"Jul 01, 2014 12:00:00 AM","Mar 01, 2015 12:00:00 AM",55B,PANORAMA RD,MOUNT WELLINGTON,Maungakiekie - Tamaki,"New (and pre-built) house, unit, bach, crib, b...",1100,140.0,200000.0,1,Residential,Detached,NaN,NaN,1998,643401.0,2013.0,Residential - Mixed Housing Suburban Zone,Residential,18.0,ACC_24,Residential 6a,Residential,NaN,NaN,NaN,174.830461,-36.904046,POINT (1763079.439 5914207.500),False,55b panorama,55b panorama rd mount wellington,4963967.0,"(55 panorama,)"
207224,151349,"Jul 01, 2014 12:00:00 AM","Mar 01, 2015 12:00:00 AM",67,CUSTOMS ST EAST,AUCKLAND CENTRAL,Waitemata,New Offices,1461,0.0,0.0,1,Business,NaN,Group 2,Offices,1998,438104.0,2013.0,Business - City Centre 

In [ ]:
def check_match(row):
    if not pd.isna(row.number_name) and not pd.isna(row.addresses):
        if row.number_name in row.addresses:
            return 1
    return np.nan
df["LINZ_MATCH_CODE"] = df.apply(check_match, axis=1)
df["LINZ_MATCH_CODE"].value_counts(dropna=False)

1.0    125254
NaN     81973
Name: LINZ_MATCH_CODE, dtype: int64